In [ ]:
import os
import mysql.connector
from mysql.connector import errors

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

# Database credentials
DB_CONNECTION = os.getenv('DB_CONNECTION')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_USERNAME = os.getenv('DB_USERNAME')
DB_PASSWORD = os.getenv('DB_PASSWORD')

# Connect to the database
try:
    conn = mysql.connector.connect(
        host=DB_HOST,
        port=DB_PORT,
        database=DB_DATABASE,
        user=DB_USERNAME,
        password=DB_PASSWORD
    )
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM foqus;")
    rows = cursor.fetchall()
    cursor.close()
    print(rows)
except errors.ProgrammingError as err:
    print(f"Error connecting to MySQL: {err}")
finally:
    conn.close()


In [ ]:
import connect as co
import os

co.connect_to_s3()

files = co.list_files_in_folder("visua")

print(files[2])

# get current working directory
cwd = os.getcwd()

file_name = "2.mp4"

co.download_file_from_s3(files[1], cwd + "/videos/2.mp4")

In [ ]:
import cv2
from detect_utils import predict, draw_boxes
import torch

model = torch.hub.load('pytorch/vision', 'resnet50', pretrained=True)
model.eval()

cap = cv2.VideoCapture('videos/1.mp4')

fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, fps, (width, height))

while True:
    # Read a frame from the video.
    ret, frame = cap.read()
    if not ret:
        break
    
    # Call the predict function to get the bounding boxes, class names, and labels.
    boxes, classes, labels = predict(frame, model, torch.device('cpu'), detection_threshold=0.5)
    
    # Draw the bounding boxes on the image.
    image = draw_boxes(boxes, classes, labels, frame)
    
    # Write the frame with bounding boxes to the output video.
    out.write(image)
    
    # Display the output image with bounding boxes.
    cv2.imshow('frame', image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    cap.release()
    out.release()
    cv2.destroyAllWindows()


In [3]:
import cv2
import numpy as np
import torch
import torchvision.transforms as transforms
from detect_utils import predict, draw_boxes
from coco_names import COCO_INSTANCE_CATEGORY_NAMES as coco_names
from model import get_model

# Define the torchvision image transforms.
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Load the pre-trained ResNet model.
model = get_model()


In [4]:
# Load the trained weights.
model.load_state_dict(torch.load('./models/model_weights.pth', map_location=torch.device('cpu')))

model.eval()

# Define the input and output video paths.
input_path = 'videos/1.mp4'
output_path = 'output.mp4'

# Open the input video file.
cap = cv2.VideoCapture(input_path)

# Get the input video's frame rate and dimensions.
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create a VideoWriter object to write the output video.
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

frame_counter = 0  # Add a frame counter

# Loop over each frame in the input video.
while True:
    # Read a frame from the input video.
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_counter += 1  # Increment the frame counter
    if frame_counter % 1000 != 0:  # Skip frames if the frame_counter is not a multiple of 1000
        continue
    
    # Call the predict function to get the bounding boxes, class names, and labels.
    boxes, classes, labels = predict(frame, model, torch.device('cpu'), detection_threshold=0.3)
    
    good_labels = []

    for label in labels:
        good_labels.append(label-1)
        print(label)

    # Draw the bounding boxes on the image.
    image = draw_boxes(boxes, classes, good_labels, frame)
    
    # Write the frame with bounding boxes to the output video.
    out.write(image)
    
    # Display the output image with bounding boxes.
    cv2.imshow('frame', image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the VideoCapture and VideoWriter objects, and close the OpenCV window.
cap.release()
out.release()
cv2.destroyAllWindows()


[28 17 17  4 17  4  3  4 32 28]
tensor(28)
tensor(17)
tensor(17)
tensor(4)
tensor(17)
tensor(4)
tensor(3)
tensor(4)
tensor(32)
tensor(28)
[17 28 17  4  4  3 32  4 17 28 17]
tensor(17)
tensor(28)
tensor(17)
tensor(4)
tensor(4)
tensor(3)
tensor(32)
tensor(4)
tensor(17)
tensor(28)
tensor(17)
[17  4  4  4 17 17 32 21 16]
tensor(17)
tensor(4)
tensor(4)
tensor(4)
tensor(17)
tensor(17)
tensor(32)
tensor(21)
tensor(16)
[28 17  4  4 17  4  3 17 32 21 28 16]
tensor(28)
tensor(17)
tensor(4)
tensor(4)
tensor(17)
tensor(4)
tensor(3)
tensor(17)
tensor(32)
tensor(21)
tensor(28)
tensor(16)
[17 28  4 17  4  4 17  3 32 21]
tensor(17)
tensor(28)
tensor(4)
tensor(17)
tensor(4)
tensor(4)
tensor(17)
tensor(3)
tensor(32)
tensor(21)
[28 17 17  4  4  4 17  3 32 21]
tensor(28)
tensor(17)
tensor(17)
tensor(4)
tensor(4)
tensor(4)
tensor(17)
tensor(3)
tensor(32)
tensor(21)
[28 17  4  4  3  4 17 17 32 21]
tensor(28)
tensor(17)
tensor(4)
tensor(4)
tensor(3)
tensor(4)
tensor(17)
tensor(17)
tensor(32)
tensor(21)
[17 2

: 

In [ ]:
import torch
import cv2
from utils import (
    collate_fn,
    get_transform,
    myOwnDataset,
)
import config
from eval import evaluate_model
import model

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Load the PyTorch model
model =  model.get_model(device=device, model_name='v2')
model.load_state_dict(torch.load('models/bigData.pth', map_location=torch.device('cpu')))

# # Create test Dataset
# testing_dataset = myOwnDataset(
#     root=config.data_dir, annotation=config.test_coco, transforms=get_transform()
# )

# # Testing DataLoader
# testing_loader = torch.utils.data.DataLoader(
#     testing_dataset,
#     batch_size=config.test_batch_size,
#     shuffle=config.train_shuffle_dl,
#     num_workers=config.num_workers_dl,
#     collate_fn=collate_fn,
# )


# # Evaluate the model
# output = evaluate_model(model.to(device), device, testing_loader, "notrans")

# # Print the output
# print(output)
